In [8]:
# https://github.com/langchain-ai/langchain/issues/9717
# https://python.langchain.com/docs/use_cases/question_answering/

import os
import sys
from config import GOOGLE_AI_API_KEY

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import GooglePalmEmbeddings
from langchain.vectorstores import Chroma

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate



In [9]:
# Document loading
path = './case_files/H1B-Process.txt'
loader = TextLoader(path)
data = loader.load()

In [10]:
# Text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [11]:
all_splits[10].metadata

{'source': './case_files/H1B-Process.txt'}

In [12]:
# Create retriever
vectorstore = Chroma.from_documents(documents = all_splits, embedding=GooglePalmEmbeddings(google_api_key = GOOGLE_AI_API_KEY))


In [6]:
from langchain.prompts import ChatPromptTemplate

with open('./questions.txt', mode='r') as questions_file:
    list_questions = questions_file.readlines()

prompt = """For the below given context answer these questions. 
If the information to answer the question is not present in the context say "I dont know". 
Answer as precise and concise as possible. If you need more information from the user to answer the question. Ask the question.
Check the elligibility for the user and ask a followup question.

To be eligible for an H1B visa, you must:

* Have a bachelor's degree or its equivalent in a specific field of study.
* Have a job offer from a U.S. employer in a specialty occupation that requires a bachelor's degree or its equivalent.
* Be a member of the professions listed in the U.S. Department of Labor's Occupational Outlook Handbook.
* Be able to speak, read, and write English.
* Not be inadmissible to the United States.


Consider below example


Context: You should be atleast 160 cm tall to ride this bus.
Question: Can I ride the bus
AI: How tall are you?

Context: {context}
Questions:{questions}"
"""

promt_template_question = ChatPromptTemplate.from_template(prompt)

In [13]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

llm = ChatGooglePalm(google_api_key=GOOGLE_AI_API_KEY, temperature=0.0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": vectorstore.as_retriever() | format_docs, "questions": RunnablePassthrough()}
    | promt_template_question
    | llm
    | StrOutputParser()
)

In [14]:
rag_chain.invoke("Can I ride the bus?")

"To be eligible for an H1B visa, you must:\n\n* Have a bachelor's degree or its equivalent in a specific field of study.\n* Have a job offer from a U.S. employer in a specialty occupation that requires a bachelor's degree or its equivalent.\n* Be a member of the professions listed in the U.S. Department of Labor's Occupational Outlook Handbook.\n* Be able to speak, read, and write English.\n* Not be inadmissible to the United States.\n\nIf you meet all of these requirements, you may be eligible for an H1B visa. However, it is important to note that the H1B visa is a nonimmigrant visa, which means that you will not be able to stay in the United States permanently. You will need to renew your visa every three years, and you will eventually need to leave the United States.\n\nIf you are interested in applying for an H1B visa, you should contact a qualified immigration attorney. An attorney can help you determine if you are eligible for the visa and can assist you with the application proc

In [15]:
rag_chain.invoke("Who is the informant or complainant?")

ChatGooglePalmError: ChatResponse must have at least one candidate.

In [22]:
rag_chain.invoke("Whether the transit remand for accused was obtained??")

"The transit remand for the accused was not obtained. The accused was not selected to apply for an H1B visa this time. The accused can request premium processing at a later date by sending their form to the same service center as their original Form I129. The accused will be able to see their status in their USCIS account once they have registered. If an attorney or a representative has filed for the accused, they will be able to see their status in their account. The account will show their status as one of the following: Submitted, Selected, Not Selected, Denied, Invalidated, or Failed Payment. Once the registration period has finished, USCIS will let the accused know if they have been selected. The accused's registration must be selected if they want to apply for an H1B visa unless they are eligible for an exemption. If the accused is not selected, USCIS will let them or their representative know once the H1B cap has been reached that year."

In [45]:
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)


In [46]:
chat_history = []

query = "Am I eligible to apply for H1B?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])


To be eligible for an H1B visa, you must:

* Have a bachelor's degree or its equivalent in a specific field of study.
* Have a job offer from a U.S. employer in a specialty occupation that requires a bachelor's degree or its equivalent.
* Be a member of the professions listed in the U.S. Department of Labor's Occupational Outlook Handbook.
* Be able to speak, read, and write English.
* Not be inadmissible to the United States.

If you meet all of these requirements, you may be eligible to apply for an H1B visa. You can find more information on the U.S. Citizenship and Immigration Services (USCIS) website.

Here are some additional details about the H1B visa:

* The H1B visa is a nonimmigrant visa that allows U.S. employers to hire foreign workers in specialty occupations that require theoretical or technical expertise.
* The H1B visa is a temporary visa, and the maximum period of stay is six years.
* H1B visa holders are eligible to bring their spouses and unmarried children under the 

In [16]:
import chromadb
from chromadb.config import Settings

In [18]:
chroma_client = chromadb.HttpClient(host="localhost", port = 8000, settings=Settings(anonymized_telemetry=False, allow_reset = False))

In [21]:
collection = chroma_client.get_collection(name="case_file_collection")

In [24]:
collection.peek()

{'ids': ['1'],
 'embeddings': [[-0.05989781394600868,
   0.049618981778621674,
   -0.06704222410917282,
   -0.1076342761516571,
   -0.034497156739234924,
   0.015161062590777874,
   0.04495345801115036,
   0.12210913747549057,
   -0.0004562873800750822,
   0.0015874234959483147,
   0.11555259674787521,
   -0.047771286219358444,
   0.03951850160956383,
   -0.020092302933335304,
   0.05830855667591095,
   0.07170798629522324,
   0.10079563409090042,
   0.0035468365531414747,
   -0.04360827058553696,
   -0.04257865995168686,
   -0.003056175075471401,
   0.0218664463609457,
   -0.010406238958239555,
   -0.033660855144262314,
   0.016781294718384743,
   -0.07868801802396774,
   -0.07153864204883575,
   0.020188285037875175,
   0.06395018100738525,
   -0.044736094772815704,
   -1.2345967661531176e-05,
   0.04758802056312561,
   0.015273689292371273,
   -0.000947001448366791,
   -0.01751631870865822,
   0.027698548510670662,
   -0.02258490025997162,
   0.015054788440465927,
   0.0226219184696

In [ ]:
collection.query(query = "")